# 06: M2/S2 - Supervised Classification (Sentence Level)

**Cíl:** Natrénovat klasifikátory na úrovni celých vět.
**Hypotéza:** Věta nese více kontextu než samotné slovo. Funguje lépe průměr všech tokenů (**Mean Pooling**) nebo speciální token (**[CLS]**)?

**Scénáře:**
* **S2a - Gold Balanced:** Trénink na Gold datech (undersampling L0 na 1:1).
* **S2b - Hybrid:** Trénink na mixu Gold (L0) + Silver (L1).

**Pooling Metody:**
* **Mean:** Průměr embeddingů všech slov ve větě.
* **CLS:** Embedding speciálního tokenu [CLS] (reprezentace celé sekvence dle BERTa).

## 1. Setup & Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import logging
from pathlib import Path
import os
from itables import show

from sklearn.metrics import classification_report

# Auto-reload modules for development
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Add src to path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)


# Vlastní moduly
import config
import data_splitting
import models
import evaluation
import visualization

# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Nastavení vizualizace
visualization.setup_style()

print(f"✅ Setup complete. Results dir: {config.RESULTS_DIR}")

## 2. Data Check
Ověříme počty vět pro jednotlivé scénáře.

In [ ]:
SCENARIOS_TO_CHECK = ['baseline', 'hybrid']
POOLING = 'mean' # Počty jsou stejné pro mean i cls, liší se jen dimenze X

print(f"{'='*80}")
print(f"📊 DATA CHECK REPORT (M2/S2 - Sentence Level)")
print(f"{'='*80}")

for scenario in SCENARIOS_TO_CHECK:
    print(f"\n🔹 SCÉNÁŘ: {scenario.upper()}")
    try:
        data = data_splitting.get_train_val_test_splits(
            scenario=scenario,
            level='sentence',
            pooling=POOLING,
            random_state=42
        )
        
        def print_stats(name, y):
            n_l0, n_l1 = np.sum(y == 0), np.sum(y == 1)
            ratio = n_l0 / n_l1 if n_l1 > 0 else 0
            print(f"   {name:<6} | Total: {len(y):<5} | L0: {n_l0:<4} | L1: {n_l1:<4} | Ratio: {ratio:.1f}:1")

        print_stats("TRAIN", data['y_train'])
        print_stats("VAL",   data['y_val'])
        print_stats("TEST",  data['y_test'])
        
    except Exception as e:
        print(f"   ❌ Chyba: {e}")

## 3. Experiment Loop (Pooling & Scenarios)
Trénujeme kombinace: **Scénář x Pooling x Model**.

**Scénáře:**
* **S2a:** Baseline (Gold) + Manual Undersampling
* **S2b:** Hybrid (Gold L0 + Silver L1)

**Pooling:** `mean` vs `cls`

In [ ]:
# Cesta pro výsledky
RESULTS_PATH = config.RESULTS_DIR / "M2_S2_experiment_results_v1.csv"

# Definice experimentů (Scénáře)
SCENARIOS = [
    {'id': 'S2a', 'name': 'Gold Balanced', 'scenario': 'baseline', 'balance_train': True},
    {'id': 'S2b', 'name': 'Hybrid (G+S)',  'scenario': 'hybrid',   'balance_train': False}, # Hybrid je už balanced z modulu
]

POOLING_METHODS = ['mean', 'cls']

# MODELS_TO_TEST = ['LogReg', 'SVM (RBF)'] # Pro rychlost. SVM (Lin) můžeš přidat.          # Pro krátký test
MODELS_TO_TEST = ["LogReg", "SVM (RBF)", "XGBoost", "Dummy", "SVM (Lin)", "NaiveBayes", "RandForest"]

if models.XGBOOST_AVAILABLE:
    MODELS_TO_TEST.append('XGBoost')


In [ ]:
results = []
best_f1 = 0.0
best_run = None

print(f"🚀 STARTING SENTENCE LEVEL EXPERIMENTS (All Metrics: P/R/F1/AUC)...")
print(f"💾 Results path: {RESULTS_PATH}")

for pooling in POOLING_METHODS:
    print(f"\n{'#'*60}")
    print(f"🌊 POOLING METHOD: {pooling.upper()}")
    print(f"{'#'*60}")
    
    for exp in SCENARIOS:
        print(f"\n   🧪 SCENARIO: {exp['id']} - {exp['name']}")
        
        try:
            # 1. Načtení dat
            data = data_splitting.get_train_val_test_splits(
                scenario=exp['scenario'],
                level='sentence',
                pooling=pooling,
                random_state=42
            )
            
            X_train, y_train = data['X_train'], data['y_train']
            X_val, y_val     = data['X_val'], data['y_val']
            X_test, y_test   = data['X_test'], data['y_test']
            
            print(f"      📊 Train Size: {X_train.shape[0]} (L0: {sum(y_train==0)}, L1: {sum(y_train==1)})")

            # 2. Trénink Modelů
            for model_name in MODELS_TO_TEST:
                print(f"      ⚙️ {model_name}...")
                
                try:
                    clf = models.get_supervised_model(model_name, random_state=42)
                    clf.fit(X_train, y_train)
                    
                    # Získání skóre
                    if hasattr(clf, "predict_proba"):
                        s_train = clf.predict_proba(X_train)[:, 1]
                        s_val   = clf.predict_proba(X_val)[:, 1]
                        s_test  = clf.predict_proba(X_test)[:, 1]
                    else:
                        s_train = clf.decision_function(X_train)
                        s_val   = clf.decision_function(X_val)
                        s_test  = clf.decision_function(X_test)
                    
                    # Threshold z Val
                    threshold, _ = evaluation.find_optimal_threshold(y_val, s_val, metric='f1')
                    
                    # Výpočet metrik pro VŠECHNY sady
                    m_train = evaluation.calculate_metrics(y_train, (s_train > threshold).astype(int), s_train)
                    m_val   = evaluation.calculate_metrics(y_val, (s_val > threshold).astype(int), s_val)
                    m_test  = evaluation.calculate_metrics(y_test, (s_test > threshold).astype(int), s_test)
                    
                    # Log (Ukládáme komplet všechno)
                    res = {
                        'id': exp['id'],
                        'scenario': exp['scenario'],
                        'scenario_name': exp['name'],
                        'pooling': pooling,
                        'model': model_name,
                        'threshold': threshold,
                        
                        # Train Metrics
                        'train_f1': m_train['f1'], 'train_auprc': m_train['avg_precision'], 'train_roc_auc': m_train['roc_auc'],
                        'train_prec': m_train['precision'], 'train_rec': m_train['recall'], # <--- NOVÉ
                        
                        # Val Metrics
                        'val_f1': m_val['f1'],     'val_auprc': m_val['avg_precision'],     'val_roc_auc': m_val['roc_auc'],
                        'val_prec': m_val['precision'], 'val_rec': m_val['recall'],         # <--- NOVÉ
                        
                        # Test Metrics
                        'test_f1': m_test['f1'],   'test_auprc': m_test['avg_precision'],   'test_roc_auc': m_test['roc_auc'],
                        'test_prec': m_test['precision'], 'test_rec': m_test['recall']      # <--- NOVÉ
                    }
                    results.append(res)
                    pd.DataFrame(results).to_csv(RESULTS_PATH, index=False)
                    
                    # Best Run Check (Stále podle F1 na testu)
                    if m_test['f1'] > best_f1:
                        best_f1 = m_test['f1']
                        best_run = {
                            'info': res,
                            'model': clf,
                            'data': data,
                            'scores_test': s_test,
                            'y_test': y_test
                        }
                        
                except Exception as e:
                    print(f"      ❌ Error {model_name}: {e}")
                    
        except Exception as e:
            print(f"   ❌ Error loading data: {e}")

print("\n✅ All experiments finished.")

## 4. Results Overview
Srovnání vlivu Poolingu a Scénářů.

In [ ]:
# =============================================================================
# 4. RESULTS OVERVIEW
# =============================================================================

# Načtení výsledků
if RESULTS_PATH.exists():
    df_results = pd.read_csv(RESULTS_PATH)
else:
    df_results = pd.DataFrame(results)

# -----------------------------------------------------------------------------
# A) PIVOT TABLE (Tabulka výsledků)
# -----------------------------------------------------------------------------
print("📊 SROVNÁNÍ AUPRC (Pooling x Scenario):")

pivot = df_results.pivot_table(
    values='test_auprc', 
    index='model',        
    columns=['pooling', 'scenario_name'], # Vytvoří MultiIndex ve sloupcích
    aggfunc='max'
)

# ✅ OPRAVA SORTINGU PRO MULTIINDEX
# Vypočítáme průměr pro každý řádek (model) bokem
mean_scores = pivot.mean(axis=1)
# Získáme seřazený seznam modelů
sorted_models = mean_scores.sort_values(ascending=False).index
# Přeřadíme tabulku podle tohoto seznamu
pivot = pivot.reindex(sorted_models)

# Zobrazíme (bez sloupce mean, ten byl jen pro řazení)
display(pivot.style.background_gradient(cmap='Blues', axis=None).format("{:.4f}"))

# -----------------------------------------------------------------------------
# B) VIZUALIZACE (Pooling Breakdown)
# -----------------------------------------------------------------------------
print("\n📊 DETAILNÍ GRAFY: MEAN vs CLS (Train vs Test)")

# Příprava dat (kopie)
df_viz = df_results.copy()

# Seznam metrik, které chceme vykreslit
metrics_to_plot = ['auprc', 'f1',] 

for m in metrics_to_plot:
    # Kontrola, zda máme data (např. 'test_prec')
    if f'test_{m}' in df_viz.columns:
        print(f"   Generating plot for: {m.upper()}...")
        
        visualization.plot_pooling_breakdown(
            df_viz, 
            metric=m,
            save_path=config.RESULTS_DIR / f"M2_S2_Pooling_Breakdown_{m.upper()}.png"
        )
    else:
        print(f"⚠️ Metrika '{m}' v datech chybí.")

print("\n✅ Vizualizace hotova.")

## 5. Deep Dive: Winner Analysis
Detailní pohled na nejlepší model.

In [ ]:
# 5. DEEP DIVE: WINNER ANALYSIS (Retrain & Metrics)

if RESULTS_PATH.exists():
    df_results = pd.read_csv(RESULTS_PATH)
    best_row = df_results.sort_values('test_f1', ascending=False).iloc[0]
    
    print(f"🏆 WINNER: {best_row['model']} ({best_row['scenario_name']})")
    print(f"🌊 Pooling: {best_row['pooling'].upper()}")
    print(f"📊 F1 (Test): {best_row['test_f1']:.4f}")
    print(f"🎯 Threshold: {best_row['threshold']:.4f}")
    print("-" * 60)
    
    # 1. Reload Data
    print(f"🔄 Reloading data...")
    data_best = data_splitting.get_train_val_test_splits(
        scenario=best_row['scenario'],
        level='sentence',
        pooling=best_row['pooling'],
        random_state=42
    )
    
    X_train_b, y_train_b = data_best['X_train'], data_best['y_train']
    X_val_b,   y_val_b   = data_best['X_val'],   data_best['y_val']
    X_test_b,  y_test_b  = data_best['X_test'],  data_best['y_test']
    
    # 2. Retrain Model (na plných datech, bez manual undersamplingu)
    print(f"⚙️ Retraining {best_row['model']} on full train set ({len(X_train_b)} samples)...")
    clf = models.get_supervised_model(best_row['model'], random_state=42)
    clf.fit(X_train_b, y_train_b)
    
    # 3. Získání skóre pro všechny sady
    if hasattr(clf, "predict_proba"):
        s_train = clf.predict_proba(X_train_b)[:, 1]
        s_val   = clf.predict_proba(X_val_b)[:, 1]
        s_test  = clf.predict_proba(X_test_b)[:, 1]
    else:
        s_train = clf.decision_function(X_train_b)
        s_val   = clf.decision_function(X_val_b)
        s_test  = clf.decision_function(X_test_b)
        
    # 4. Aplikace prahu (používáme ten uložený z CSV)
    thresh = best_row['threshold']
    p_train = (s_train > thresh).astype(int)
    p_val   = (s_val > thresh).astype(int)
    p_test  = (s_test > thresh).astype(int)
    
    # 5. Detailní Report Metrik
    print(f"\n📢 DETAILED PERFORMANCE REPORT (Threshold: {thresh:.4f})")
    print("=" * 60)
    
    def print_metrics(name, y_true, y_pred, scores):
        m = evaluation.calculate_metrics(y_true, y_pred, scores)
        print(f"\n🔹 {name} SET Results ({len(y_true)} samples):")
        print(f"   F1 Score:  {m['f1']:.4f}")
        print(f"   AUPRC:     {m['avg_precision']:.4f}")
        print(f"   ROC AUC:   {m['roc_auc']:.4f}")
        print(f"   Precision: {m['precision']:.4f}")
        print(f"   Recall:    {m['recall']:.4f}")
        print("-" * 40)
        print(classification_report(y_true, y_pred, target_names=['Neutral (L0)', 'Bias (L1)'], digits=4))

    print_metrics("TRAIN", y_train_b, p_train, s_train)
    print_metrics("VAL",   y_val_b,   p_val,   s_val)
    print_metrics("TEST",  y_test_b,  p_test,  s_test)
    
    # 6. Uložení do proměnné pro další buňky (Vizualizace)
    best_run = {
        'info': best_row.to_dict(),
        'model': clf,
        'data': data_best,
        'scores_test': s_test,
        'y_test': y_test_b,
        'y_pred_test': p_test, # Přidal jsem i binární predikce
        'threshold': thresh
    }

else:
    print("⚠️ Nemám výsledky. Spusť nejprve trénovací smyčku.")

In [ ]:

# 2. Vizualizace
visualization.plot_confusion_matrix_heatmap(y_test_b, y_pred, normalize=True, title="CM (Normalized)")
visualization.plot_pr_curve(y_test_b, scores_test, title="PR Curve")
visualization.plot_model_calibration(y_test_b, scores_test, title="Calibration")

#### 6. Projekce Embeddingů


In [ ]:
print("🎨 Počítám projekce vět...")
projs, idxs = visualization.compute_projections(X_test_b, methods=['PCA', 't-SNE'], random_state=42)
y_viz = y_test_b[idxs]
y_pred_viz = y_pred[idxs]

for m, coords in projs.items():
    # GT
    visualization.plot_embedding_projection(
        coords, pd.Series(y_viz).map({0:'Neutral', 1:'Bias'}), 
        palette={'Neutral': config.COLORS['l0'], 'Bias': config.COLORS['l1']},
        title=f"{m} - Ground Truth"
    )
    # Errors
    visualization.plot_error_analysis_projection(
        coords, y_viz, y_pred_viz, method_name=m
    )

## 6. Qualitative Analysis (Error Analysis)

In [ ]:
# =============================================================================
# 6. QUALITATIVE ANALYSIS (Detailní pohled na chyby)
# =============================================================================

# 1. Vezmeme existující metadata (obsahují document_id, sentence_id, text...)
# Použijeme .copy(), abychom neovlivnili původní data
df_qual = data_best['meta_test'].copy()

# 2. Přidáme predikce modelu
df_qual['true_label'] = y_test_b
df_qual['pred_label'] = y_pred
df_qual['anomaly_score'] = scores_test

# 3. Určení kategorie (TP, TN, FP, FN)
conds = [
    (df_qual['true_label'] == 1) & (df_qual['pred_label'] == 1), # TP
    (df_qual['true_label'] == 0) & (df_qual['pred_label'] == 0), # TN
    (df_qual['true_label'] == 0) & (df_qual['pred_label'] == 1), # FP
    (df_qual['true_label'] == 1) & (df_qual['pred_label'] == 0)  # FN
]

df_qual['category'] = np.select(conds, ['TP', 'TN', 'FP', 'FN'], default='UNKNOWN')

# 4. Úklid sloupců (seřadíme je logicky)
# Definujeme preferované pořadí
preferred_cols = ['document_id', 'sentence_id', 'text', 'true_label', 'pred_label', 'anomaly_score', 'category']

# Vybereme jen ty, které v datech skutečně máme (aby to nepadalo, kdyby 'sentence_id' chybělo)
final_cols = [c for c in preferred_cols if c in df_qual.columns]

# Pokud zbyly nějaké další sloupce v metadatech, přidáme je nakonec
remaining_cols = [c for c in df_qual.columns if c not in final_cols]
df_qual = df_qual[final_cols + remaining_cols]

# 5. Výpis výsledků
print("\n❌ TOP 5 FP (Model vidí bias, kde není - False Alarm):")
display(df_qual[df_qual['category'] == 'FP'].sort_values('anomaly_score', ascending=False).head(5))

print("\n❌ TOP 5 FN (Model přehlédl bias - Missed Detection):")
display(df_qual[df_qual['category'] == 'FN'].sort_values('anomaly_score', ascending=True).head(5))

# 6. Uložení
save_path = config.RESULTS_DIR / "M2_S2_Qualitative_Analysis.csv"
df_qual.to_csv(save_path, index=False)
print(f"\n💾 Detailní analýza uložena: {save_path}")